# Identification of a Neural-Network for Constitutive Law

## Preliminary part - Introduction

To allow inline pictures, run the following block

In [ ]:
%matplotlib inline

Import all the useful libraries before first run
We need here the classic ones such as:
- math
- numpy
- pandas
- matplotlib

And for the Neural Network, we also need to import parts of the keras module of TensorFlow

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Importation de TensorFlow
import tensorflow as tf
from keras.models import Sequential
from keras.metrics import MeanSquaredError
from keras.layers.core import Dense, Activation
from keras import optimizers

**Global definitions**

Here after, we define the colors to use for plotting.

In [ ]:
saveFigures = True
colorlist = ['#bb0000', '#00bb00', "#0000bb", '#bbbb00', '#bb00bb', "#00bbbb", '#bbbbbb', '#770000', '#007700', "#000077", '#777700', '#770077', "#007777", '#777777', '#440000', '#004400', "#000044", '#444400', '#440044', "#0044444", '#444444','#000000']

Useful functions to define the number of subplots to use

In [ ]:
baseSize = (8, 6)  # Base size of a subplot

def sbPlot(n):
    if (n == 1): return 1, 1
    if (n <= 2): return 1, 2
    if (n <= 4): return 2, 2
    if (n <= 6): return 3, 2
    if (n <= 9): return 3, 3
    if (n <= 12): return 4, 3
    return 0, 0

def sbPlotSize(n):
    x, y = sbPlot(n)
    return baseSize[0] * y, baseSize[1] * x

Used to plot an Histogram

In [ ]:
def getDigitsFrom(n):
    d = int(math.log10(abs(n)))
    return n*10**(-d + 1), d - 1

def plotHisto(Y, labels, bottom, top, xlab, ylab, figname, title):
    numb = len(Y)
    X = np.arange(numb) +1

    plt.figure(figsize = (12, 9)) # for a4 landscape
    plt.rc('text', usetex=True)
    plt.rcParams['xtick.labelsize'] = 16
    plt.rcParams['ytick.labelsize'] = 16

    plt.bar(X, Y, color = '#770000')

    for x, y in zip(X, Y):
        u,v = getDigitsFrom(y)
        plt.text(x, y + 0.01*(top-bottom), '$%.2f \\times 10^{%d}$' % (u,v), ha='center', va='bottom', fontsize=16)

    for i in range(0,numb):
        if (Y[i]> 0.3*top):
            plt.text(X[i], (top-bottom)/10 + bottom , labels[i], ha='center', va='bottom', fontsize=20, rotation=90, color='white')
        else:
            plt.text(X[i], 5*(top-bottom)/10 + bottom , labels[i], ha='center', va='bottom', fontsize=20, rotation=90, color='black')

    plt.ylim(bottom, top)

    plt.ylabel(ylab, fontsize = 20)
    plt.xlabel(xlab, fontsize = 20)
    plt.title(title, fontsize = 20)

    plt.tick_params(axis='x',top=False, bottom=False, left=False, right=False,
                labelleft=False, labelbottom=False)
    plt.grid(True)
    if (saveFigures) : plt.savefig(figname)
    else: plt.show()

## Definition of the data file to use 
We define here after the directory and the Excel Datafile used for the Neural-Network construction.

In [ ]:
# dataPath = 'ANN-Zhou-Law'
# dataFile = 'ExperimentsTreated.xlsx'

In [ ]:
dataPath = 'ANN-JohnsonCook'
dataFile = 'JC-Experiments.xlsx'

## Open the Excel file and import data
Read the content of the Excel file using the pandas library method read_excel.

In this datafile, temperatures must be on separate sheets, while the corresponding sheets contains stress / strain / strain-rate values: $$\overline{\sigma}=f({\overline{\varepsilon}^p},\dot{\overline{\varepsilon}^p})$$

In [ ]:
excelData = pd.read_excel(dataPath + '/' + dataFile, None)

Extract the list of temperatures $T$ from the the Excel file.

In [ ]:
len(excelData)
temperatures = []
temps = list(excelData.keys())
for T in temps:
    temperatures.append(int(T.replace('°C', '')))
temperaturesOrig = temperatures
nTemp = len(temperatures)
nTemp, temperatures

Extract the list of plastic strain rates $\dot{\overline{\varepsilon}^p}$ from the the Excel file.

In [ ]:
dataSheet = excelData[str(temperatures[0]) + '°C']
epsp = []
for i in range(1, dataSheet.shape[1]):
    epsp.append(dataSheet.columns[i])
epspOrig = epsp
nEpsp = len(epsp)
nEpsp, epsp

Extract all data from the panda sheets.

In [ ]:
# Read first sheet
dataSheet = excelData[str(temperatures[0]) + '°C']
data = dataSheet.values
eps = data[:, 0]
sig = data[:, 1:]
# Append the other sheets
for T in temperatures[1:]:
    dataSheet = excelData[str(T) + '°C']
    data = dataSheet.values
    sig = np.append(sig, data[:, 1:], axis=1)
nEps = len(eps)

Creates two arrays for temperatures $T$ and plastic strain rates $\dot{\overline{\varepsilon}^p}$ and transforms the values of the plastic strain rates $\dot{\overline{\varepsilon}^p}$ into its logarithmic part.

**It is very important** to transform the epsp data into its logarithmic part since entries of the ANN show enhance a linear behavior.

In [ ]:
temperatures = np.array(temperatures)
epsp = np.array(epsp)
epsLogBase = epsp.min()
epsp = np.log(epsp / epsLogBase)
epspArray = np.tile(epsp, temperatures.shape[0])
temperaturesArray = temperatures.repeat(epsp.shape[0])
epspArray.shape, epspArray, temperaturesArray

Transform all data into an array of 4 columns where:
- $\overline{\varepsilon}^p$ : is in first column
- $\dot{\overline{\varepsilon}^p}$ : is in second column
- $T$ : is in third column
- $\overline{\sigma}$ : is th fourth column

In [ ]:
entries = np.empty([epspArray.shape[0] * eps.shape[0], 4])
row = 0
colEps = 0
colEpsp = 1
colT = 2
colSig = 3
for i in range(epspArray.shape[0]):
    for j in range(eps.shape[0]):
        entries[row, colEps] = eps[j]
        entries[row, colEpsp] = epspArray[i]
        entries[row, colT] = temperaturesArray[i]
        entries[row, colSig] = sig[j, i]
        row += 1
entries.shape, entries

Then, we must normalize all the entries, so that they vary within the range $[0:1]$.

- We compute the minEntries and rangeEntries variable that contains all the min and max Values
- We normalize the data by dividing all variables using: $$X_n = \frac{X - X_{min}}{X_{max} - X_{min}}$$

To rebuid data, we must use: $$ X = X_n(X_{max} - X_{min}) + X_{min}$$

In [ ]:
minEntries = entries.min(axis=0)
maxEntries = entries.max(axis=0)
rangeEntries = maxEntries - minEntries
NNentries = (entries - minEntries) / rangeEntries

To check the previous process precision, and lost of data

In [ ]:
print("Max error for normalized data is :",(NNentries * rangeEntries + minEntries - entries).max())

## Plot the original data

Plots the original data where subplots are dependents on the temperature $T$

In [ ]:
plt.figure(figsize = sbPlotSize(nTemp))
plt.rc('text', usetex = True)
plt.subplots_adjust(hspace = 0.3)
idx = 1
for T in temperatures:
    xs, ys = sbPlot(nTemp)
    plt.subplot(xs, ys, idx)
    cl = 0
    for epspv, epspvv in zip(epsp, epspOrig):
        # filter on T
        data = NNentries[(entries[:,colT]==T) & (entries[:,colEpsp]==epspv), :]
        # Plot the curves
        plt.plot(data[:,colEps]*rangeEntries[colEps]+minEntries[colEps], data[:,colSig]*rangeEntries[colSig]+minEntries[colEps], color=colorlist[cl], label=r'$\dot{\overline{\varepsilon}^p}=' + str(epspvv) + '\ s^{-1}$', marker='o', linewidth = 3)
        cl += 1
    plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 10)
    plt.grid()
    plt.xlabel(r'$Deformation\ \overline{\varepsilon}^{p}$', fontsize = 16)
    plt.ylabel(r'$Stress\ \overline{\sigma}$', fontsize = 16)
    plt.title(r'$T=' + str(T) + '^{\circ}C$', fontsize = 16)
    idx += 1
if (saveFigures) : plt.savefig(dataPath + '/' + 'OriginalData-T.svg', bbox_inches = 'tight', pad_inches = 0)
else : plt.show()

Plots the original data where subplots are dependents on the plastic strain rate $\dot{\overline{\varepsilon}^p}$

In [ ]:
plt.figure(figsize = sbPlotSize(nEpsp))
plt.rc('text', usetex = True)
plt.subplots_adjust(hspace = 0.3)
idx = 1
for epspv, epspvv in zip(epsp, epspOrig):
    xs, ys = sbPlot(nEpsp)
    plt.subplot(xs, ys, idx)
    cl = 0
    for T, TOrig in zip(temperatures, temperaturesOrig):
        # filter on T
        data = NNentries[(entries[:,colT]==T) & (entries[:,colEpsp]==epspv), :]
        # Plot the curves
        plt.plot(data[:,colEps]*rangeEntries[colEps]+minEntries[colEps], data[:,colSig]*rangeEntries[colSig]+minEntries[colSig], color=colorlist[cl], label=r'$T=' + str(TOrig) + '^{\circ}C$', marker = 'o', linewidth = 3)
        cl += 1
    plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 10)
    plt.grid()
    plt.xlabel(r'$Deformation\ \overline{\varepsilon}^{p}$', fontsize = 16)
    plt.ylabel(r'$Stress\ \overline{\sigma}$', fontsize = 16)
    plt.title(r'$\dot{\overline{\varepsilon}^p}=' + str(epspvv) + '\ s^{-1}$', fontsize = 16)
    idx += 1
if (saveFigures) : plt.savefig(dataPath + '/' + 'OriginalData-epsp.svg', bbox_inches = 'tight', pad_inches = 0)
else: plt.show()

## Data form transformation for the ANN

Separate input and output from data.

Input contains:
- $\overline{\varepsilon}^p$ : is in first column
- $\dot{\overline{\varepsilon}^p}$ : is in second column
- $T$ : is in third column

Output contains:
- $\overline{\sigma}$ : the only column

In [ ]:
NNinput = NNentries[:,colEps:colSig]
NNoutput = NNentries[:,colSig]
NNinput.shape, NNoutput.shape

## Neural Network set-up

In [ ]:
models = []  # The list of models to use
convergenceDatas = []   # To store convergence curves

# FACT = ['tanh', 'sigmoid']
# COUCH = [5, 7, 9, [5,3], [7,4], [9,5]]

FACT = ['sigmoid']
COUCH = [[15, 7]]

for f in FACT:
    for c in COUCH:
        desc = '3'
        model = Sequential()
        if type(c) == list:
            fst = True
            for k in c:
                if (fst): model.add(Dense(k, input_dim = 3, activation = f))
                else: model.add(Dense(k, activation = f))
                fst = False
                desc += '-' + str(k)
        else:
            model.add(Dense(c, input_dim = 3, activation = f))
            desc += '-' + str(c)
        model.add(Dense(1))
        desc += '-1-' + f
        model._name = desc
        models.append(model)
        convergenceDatas.append([desc, np.array([])])

In [ ]:
# #opt0 = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# opt0 = optimizers.Adam()
# opt1 = optimizers.Adam()
# opt2 = optimizers.Adam()
# opt3 = optimizers.Adam(amsgrad=True)

# models[0].compile(loss = 'mean_squared_error', optimizer = opt0)
# models[1].compile(loss = 'mean_squared_error', optimizer = opt1, metrics = ['accuracy'])
# models[2].compile(loss = 'mean_squared_error', optimizer = opt2, metrics = ['mean_squared_error'])
# models[3].compile(loss = 'mean_squared_error', optimizer = opt3)
# models[0]._name = models[0].name + "-none"
# models[1]._name = models[1].name + "-accuracy"
# models[2]._name = models[2].name + "-mse"
# models[3]._name = models[3].name + "-amsgrad"

Initialize models and apply solver and optimizer for each one.

Here, we choose the **adam** solver and a **mse** error.

In [ ]:
for model in models:
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()

In [ ]:
# Test code, I don't even remember
# # Load the test Database
# testFileName = dataPath + '/Datatest.npz'
# testDatabase = np.load(testFileName)
# testData = testDatabase['testData']

# # Number of rows of the test database
# nrows = testData.shape[0]

# # Extract data from the database
# eps_test = testData[:,0].reshape(nrows,1)
# epsp_test = testData[:,1].reshape(nrows,1)
# T_test = testData[:,2].reshape(nrows,1)
# sig_test = testData[:,3].reshape(nrows,1)

# # Computes the log of epsp
# epsp_test = np.log(epsp_test/epsLogBase)

# # Creates the input test data with scale
# inputTest = (np.hstack([eps_test, epsp_test, T_test]) - np.array(minEntries)[0:3]) / np.array(rangeEntries)[0:3]
# sig_test = (sig_test- np.array(minEntries)[3]) / np.array(rangeEntries)[3]
# inputTest, sig_test

Save all internal arrays for direct use of the NN

In [ ]:
def saveInternalMatrices(ANN, filename):
    if len(ANN.layers) == 1: 
        np.savez(filename,
                 logBase = np.array([epsLogBase]),
                 minEntries = minEntries, 
                 maxEntries = maxEntries,
                 w1 = ANN.layers[0].get_weights()[0].T, 
                 b1 = ANN.layers[0].get_weights()[1].reshape(len(ANN.layers[0].get_weights()[1]),1))
    if len(ANN.layers) == 2: 
        np.savez(filename, 
                 logBase = np.array([epsLogBase]),
                 minEntries = minEntries, 
                 maxEntries = maxEntries,
                 w1 = ANN.layers[0].get_weights()[0].T, 
                 b1 = ANN.layers[0].get_weights()[1].reshape(len(ANN.layers[0].get_weights()[1]),1),
                 w2 = ANN.layers[1].get_weights()[0].T, 
                 b2 = ANN.layers[1].get_weights()[1].reshape(len(ANN.layers[1].get_weights()[1]),1))
    if len(ANN.layers) == 3: 
        np.savez(filename, 
                 logBase = np.array([epsLogBase]),
                 minEntries = minEntries, 
                 maxEntries = maxEntries,
                 w1 = ANN.layers[0].get_weights()[0].T, 
                 b1 = ANN.layers[0].get_weights()[1].reshape(len(ANN.layers[0].get_weights()[1]),1),
                 w2 = ANN.layers[1].get_weights()[0].T, 
                 b2 = ANN.layers[1].get_weights()[1].reshape(len(ANN.layers[1].get_weights()[1]),1),
                 w3 = ANN.layers[2].get_weights()[0].T, 
                 b3 = ANN.layers[2].get_weights()[1].reshape(len(ANN.layers[2].get_weights()[1]),1)) 
    if len(ANN.layers) == 4: 
        np.savez(filename, 
                 logBase = np.array([epsLogBase]),
                 minEntries = minEntries, 
                 maxEntries = maxEntries,
                 w1 = ANN.layers[0].get_weights()[0].T, 
                 b1 = ANN.layers[0].get_weights()[1].reshape(len(ANN.layers[0].get_weights()[1]),1),
                 w2 = ANN.layers[1].get_weights()[0].T, 
                 b2 = ANN.layers[1].get_weights()[1].reshape(len(ANN.layers[1].get_weights()[1]),1),
                 w3 = ANN.layers[2].get_weights()[0].T, 
                 b3 = ANN.layers[2].get_weights()[1].reshape(len(ANN.layers[2].get_weights()[1]),1),
                 w4 = ANN.layers[3].get_weights()[0].T, 
                 b4 = ANN.layers[3].get_weights()[1].reshape(len(ANN.layers[3].get_weights()[1]),1))             

## Solve the parameters of all ANN models

In [ ]:
iterationsNumber = 50 # Define the number of iteration to do
epochNumber = 1000    # Define the number of epoch for each iteration
subSave = True

for model, convergenceData in zip (models, convergenceDatas):
    print("MODEL :", model.name)
    if (subSave):
        try:
            os.mkdir(dataPath + '/' + model.name)
        except:
            pass
    for i in range(iterationsNumber):
        history = model.fit(NNinput, NNoutput, epochs = epochNumber, verbose = 0, shuffle = True)
        loss = history.history['loss']
        convergenceData[1] = np.append(convergenceData[1], loss)
        if (subSave):
            saveInternalMatrices(model, dataPath + '/' + model.name + '/ANN-' + str(i))
        print("Iteration :", i + 1, "/", iterationsNumber, '-> %8.6E' % loss[-1], "  ", end = '\r')
    print("\n")

Save the Convergence curve database

In [ ]:
for convergenceData in convergenceDatas:
    name = convergenceData[0]
    convCurve = convergenceData[1]
    np.savez(dataPath + '/CD-' + name, convCurve = convCurve)
    print("Convergence data %s saved" % (name))

Load the Convergence database

In [ ]:
# names = ['3-5-1-tanh', '3-7-1-tanh']
# convergenceDatas = []   # To store convergence curves
# for name in names:
#     NN = np.load(dataPath + '/CD-' + name + '.npz')
#     cc = [name, NN['convCurve']]
#     convergenceDatas.append(cc)

## Plot results of the optimization procedure

First, we plot the convergence curves

In [ ]:
plt.figure(figsize = (12, 9))
plt.rc('text', usetex = True)
cl = 0
for convergenceData in convergenceDatas:
    name = convergenceData[0]
    curve = convergenceData[1]
    shortCurve = curve[::10]
    x = np.linspace(0, len(curve), len(shortCurve))
    plt.plot(x, shortCurve, label = name, color=colorlist[cl], linewidth = 3)
    cl += 1
plt.grid()
plt.xlabel(r'$Iteration$', fontsize = 16)
plt.ylim(0, 5e-3)
plt.ylabel(r'$Quadratic\ Error\ \Delta E$', fontsize = 16)
plt.title(r'$Global\ convergence\ of\ the\ Neural\ Network$', fontsize = 16)
plt.legend(loc = 'upper right',fancybox = True, numpoints = 1, fontsize = 14)
if (saveFigures) : plt.savefig(dataPath + '/' + 'Convergence.svg', bbox_inches = 'tight', pad_inches = 0)
else: plt.show()

The we plot the history of all models precision

In [ ]:
plt.figure(figsize = (12, 9))
plt.rc('text', usetex = True)
convs = np.array([])
labels = []
for convergenceData in convergenceDatas:
    name = convergenceData[0]
    curve = convergenceData[1]
    lastzone = int(len(curve)/20) # 5% des points pris en compte pour la convergence globale
    convs = np.append(convs, curve[-lastzone:].mean())
    labels.append(name)
bottom = 0
top = convs.max()*1.1
ylab = r'$Quadratic\ Error\ \Delta E$'
xlab = r'$Model\ used$'
figname = dataPath + '/' + 'Precision.svg'
title = r'$Global\ precision\ of\ the\ Neural\ Network$'
plotHisto(convs, labels, bottom, top, xlab, ylab, figname, title)

Plots the results of the model where all subplots are dependents on the plastic strain rate $\dot{\overline{\varepsilon}^p}$

In [ ]:
for model in models:
    predictedOutput = model.predict(NNinput)
    plotPredicted = predictedOutput.reshape(nEpsp*nTemp, nEps).T
    pcol = 0
    plt.figure(figsize = sbPlotSize(nTemp))
    plt.rc('text', usetex = True)
    plt.subplots_adjust(hspace = 0.3)
    idx = 1
    for T, TOrig in zip(temperatures, temperaturesOrig):
        pli = 0
        xs, ys = sbPlot(nTemp)
        plt.subplot(xs, ys, idx)
        for epspv, epspvAff in zip(epsp, epspOrig):
            # filter on T
            data = NNentries[(entries[:,colT]==T) & (entries[:,colEpsp]==epspv), :]
            # Plot the curves
            plt.plot(data[:,colEps]*rangeEntries[colEps]+minEntries[colEps], data[:,colSig]*rangeEntries[colSig]+minEntries[colSig], color=colorlist[pli], label=r'$\dot{\overline{\varepsilon}^p}=' + str(epspvAff) + '\ s^{-1}$', marker = 'o', linestyle='none')
            plt.plot(data[:,colEps]*rangeEntries[colEps]+minEntries[colEps], plotPredicted[:,pcol*nEpsp+pli]*rangeEntries[colSig]+minEntries[colSig], colorlist[pli], linewidth = 3)
            pli += 1
        pcol += 1
        plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
        plt.grid()
        plt.xlabel(r'$Deformation\ \overline{\varepsilon}^{p}$', fontsize = 16)
        plt.ylabel(r'$Stress\ \overline{\sigma}$', fontsize = 16)
        plt.title(r'$T=' + str(TOrig) + '^{\circ}C$', fontsize = 16)
        idx += 1
    if (saveFigures) : plt.savefig(dataPath + '/' + 'NN-' + model.name + '-epsp.svg', bbox_inches = 'tight', pad_inches = 0)
    else: plt.show()

Plots the results of the model where all subplots are dependents on the temperature $T$

In [ ]:
for model in models:
    predictedOutput = model.predict(NNinput)
    plotPredicted = predictedOutput.reshape(nEpsp*nTemp, nEps).T
    pcol = 0
    plt.figure(figsize = sbPlotSize(nEpsp))
    plt.rc('text', usetex = True)
    plt.subplots_adjust(hspace = 0.3)
    idx = 1
    for epspv, epspvAff in zip(epsp, epspOrig):
        pli = 0
        xs, ys = sbPlot(nEpsp)
        plt.subplot(xs, ys, idx)
        for T, TOrig in zip(temperatures, temperaturesOrig):
            # filter on T
            data = NNentries[(entries[:,colT]==T) & (entries[:,colEpsp]==epspv), :]
            # Plot the curves
            plt.plot(data[:,colEps]*rangeEntries[colEps]+minEntries[colEps], data[:,colSig]*rangeEntries[colSig]+minEntries[colSig], color=colorlist[pli], label=r'$T=' + str(TOrig) + '^{\circ}C$', marker = 'o', linestyle='none')
            plt.plot(data[:,colEps]*rangeEntries[colEps]+minEntries[colEps], plotPredicted[:,pcol+nEpsp*pli]*rangeEntries[colSig]+minEntries[colSig], colorlist[pli], linewidth = 3)
            pli += 1
        pcol += 1
        plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
        plt.grid()
        plt.xlabel(r'$Deformation\ \overline{\varepsilon}^{p}$', fontsize = 16)
        plt.ylabel(r'$Stress\ \overline{\sigma}$', fontsize = 16)
        plt.title(r'$\dot{\overline{\varepsilon}^p}=' + str(epspvAff) + '\ s^{-1}$', fontsize = 16)
        idx += 1
    if (saveFigures) : plt.savefig(dataPath + '/' + 'NN-' + model.name + '-T.svg', bbox_inches = 'tight', pad_inches = 0)
    else: plt.show()

## Save results for future use

Save the Tensorflow model

In [ ]:
for model in models:
    model.save(dataPath + '/' + model.name + '/model')

Load the Tensorflow models

In [ ]:
# models = []              # The list of models to use
# modNames = ['3-15-7-1-sigmoid']
# for modName in modNames:
#     new_model = tf.keras.models.load_model(dataPath + '/' + modName + '/model')
#     models.append(new_model)

Saves the internal matrices data for future use

In [ ]:
for model in models:
    saveInternalMatrices(model, dataPath + '/' + model.name)

## Some statistics

In [ ]:
for model in models:
    predictedOutput = model.predict(NNinput).T
    #np.mean((NNoutput-predictedOutput)**2)
    num = np.sum((NNoutput-np.mean(NNoutput))*(predictedOutput-np.mean(predictedOutput)))
    den = math.sqrt(np.sum((NNoutput-np.mean(NNoutput))**2)*np.sum((predictedOutput-np.mean(predictedOutput))**2))
    R = num/den
    #np.mean(np.abs(((NNoutput+1)-(predictedOutput+1))/(NNoutput+1)))*100
    plt.figure(figsize = (12, 8))
    plt.rc('text', usetex = True)
    plt.scatter(NNoutput* rangeEntries[3] + minEntries[3], predictedOutput* rangeEntries[3] + minEntries[3], color='red', label="$AN\!N\ prediction$")
    plt.plot([minEntries[3],rangeEntries[3] + minEntries[3]], [minEntries[3],rangeEntries[3] + minEntries[3]], color = 'black', label='$Best\ fit$')
    plt.xlim(minEntries[3], rangeEntries[3] + minEntries[3])
    plt.ylim(minEntries[3],rangeEntries[3] + minEntries[3])
    plt.grid()
    plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
    plt.xlabel(r'$Analytical\ flow\ stress\ \overline{\sigma}$', fontsize = 16)
    plt.ylabel(r'$Predicted\ flow\ stress\ \overline{\sigma}$', fontsize = 16)
    plt.text(minEntries[3] + 0.2*rangeEntries[3], minEntries[3]+ 0.7*rangeEntries[3], '$R = %.8f$' % (R), fontsize=20)
    #plt.title(r'$Map\ of\ errors\ \overline{\varepsilon}^{p} / T$', fontsize = 20)
    if (saveFigures) : plt.savefig(dataPath + '/' + 'NN-' + model.name + '-error-sig.svg', bbox_inches = 'tight', pad_inches = 0)
    else: plt.show()    

## Test  efficiency of the NN with random values

Scan the content of the testDatabase if it exists

In [ ]:
testFileName = dataPath + '/Datatest.npz'
displayError = False
import os.path
if (os.path.exists(testFileName)):
    
    # Load the test Database
    testDatabase = np.load(testFileName)
    testData = testDatabase['testData']
    
    # Number of rows of the test database
    nrows = testData.shape[0]
    
    # Extract data from the database
    eps_test = testData[:,0].reshape(nrows,1)
    epsp_test = testData[:,1].reshape(nrows,1)
    T_test = testData[:,2].reshape(nrows,1)
    sig_test = testData[:,3].reshape(nrows,1)
    
    # Computes the log of epsp
    epsp_test = np.log(epsp_test/epsLogBase)
    
    # Creates the input test data with scale
    inputTest = (np.hstack([eps_test, epsp_test, T_test]) - np.array(minEntries)[0:3]) / np.array(rangeEntries)[0:3]

    # Computes the predicted values and rescale back them
    for model in models:
        print("Model :", model.name)
        sigNN = model.predict(inputTest) * rangeEntries[3] + minEntries[3]
    
        # Computes the error
        error = np.abs(sigNN - sig_test)/sigNN
        print("Max error is:", np.max(error))
        print("Mean error is:", np.mean(error))
        errorSort = np.sort(error, axis=0)
        largeErrors = errorSort[errorSort > 5/1000]
        print(len(largeErrors),'/',len(error),'points have errors > 0.5 %\n')

        # Display info for large errors
        if (displayError):
            for err in largeErrors[::-1]:
                loc = np.where(error == err)[0][0]
                print('err= %5.3f %% for eps= %5.3E, epsp= %5.3E, T= %5.2f' %(error[loc][0]*100, eps_test[loc][0], math.exp(epsp_test[loc][0]), T_test[loc][0]))
            print('-----------------------------------------------------------------------\n')
        
        plt.figure(figsize = (12, 8))
        plt.rc('text', usetex = True)
        plt.scatter(eps_test[error >= 0.01], T_test[error >= 0.01], color='black', label="$> 1.00 \%$")
        plt.scatter(eps_test[(error < 0.01) & (error >= 0.005)], T_test[(error < 0.01) & (error >= 0.005)], color='red', label="$0.5 - 1.00 \%$")
        plt.scatter(eps_test[(error < 0.005) & (error >= 0.0025)], T_test[(error < 0.005) & (error >= 0.0025)], color='orange', label="$0.25 - 0.50 \%$")
        plt.scatter(eps_test[(error < 0.0025) & (error >= 0.001)], T_test[(error < 0.0025) & (error >= 0.001)], color='yellow', label="$0.10 - 0.25 \%$")
        plt.scatter(eps_test[error < 0.001], T_test[error < 0.001], color='green', label="$< 0.10 \%$")
        plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
        plt.xlabel(r'$Deformation\ \overline{\varepsilon}^{p}$', fontsize = 16)
        plt.ylabel(r'$T^{\circ}C$', fontsize = 16)
        plt.title(r'$Map\ of\ errors\ \overline{\varepsilon}^{p} / T$', fontsize = 20)
        if (saveFigures) : plt.savefig(dataPath + '/' + 'NN-' + model.name + '-error-T-eps.svg', bbox_inches = 'tight', pad_inches = 0)
        else: plt.show()
        
        plt.figure(figsize = (12, 8))
        plt.scatter(eps_test[error >= 0.01], np.exp(epsp_test[error >= 0.01]), color='black', label="$> 1.00 \%$")
        plt.scatter(eps_test[(error < 0.01) & (error >= 0.005)], np.exp(epsp_test[(error < 0.01) & (error >= 0.005)]), color='red', label="$0.50 - 1.0 \%$")
        plt.scatter(eps_test[(error < 0.005) & (error >= 0.0025)], np.exp(epsp_test[(error < 0.005) & (error >= 0.0025)]), color='orange', label="$0.25 - 0.50 \%$")
        plt.scatter(eps_test[(error < 0.0025) & (error >= 0.001)], np.exp(epsp_test[(error < 0.0025) & (error >= 0.001)]), color='yellow', label="$0.10 - 0.25 \%$")
        plt.scatter(eps_test[error < 0.001], np.exp(epsp_test[error < 0.001]), color='green', label="$< 0.10 \%$")
        plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
        plt.xlabel(r'$Deformation\ \overline{\varepsilon}^{p}$', fontsize = 16)
        plt.ylabel(r'$\dot{\overline{\varepsilon}^{p}}$', fontsize = 16)
        plt.title(r'$Map\ of\ errors\ \overline{\varepsilon}^{p} / \dot{\overline{\varepsilon}^{p}}$', fontsize = 20)
        if (saveFigures) : plt.savefig(dataPath + '/' + 'NN-' + model.name + '-error-epsp-eps.svg', bbox_inches = 'tight', pad_inches = 0)
        else: plt.show()